In [2]:

# install pakages  (  -> first time )

'''
!pip install urllib3
!pip install bs4
!pip install pandas
!pip install tqdm

'''

# 기본 세팅

from urllib import error
from urllib.request import Request, urlopen
from urllib.parse import quote, urlencode, quote_plus, unquote
import json
from bs4 import BeautifulSoup as bs


import copy
import os
import datetime as dt
import time as t
import numpy as np
import pandas as pd
import random as rd

from tqdm import tqdm



In [5]:
'''1. 목표 날짜 지정  (검색할 날짜)'''
# 검색 시작 날짜
first_date = 20200101





first_date_dt = dt.datetime(int(str(first_date)[:4]) , int(str(first_date)[4:6]), int(str(first_date)[6:]))
first_date_str = first_date_dt.strftime('%Y%m%d')




# 검색 종료 날짜
end_date = 20210120









end_date_dt = dt.datetime(int(str(end_date)[:4]) , int(str(end_date)[4:6]), int(str(end_date)[6:]))
end_date_str = end_date_dt.strftime('%Y%m%d')

# 검색할 날짜 수 계산 (type : datetime)
number_of_days = (end_date_dt - first_date_dt).days

In [6]:





start_time = t.time()

########################################
##### 1. url 크롤링                #####   -> '한겨레 url주소.csv' 생성
########################################

data_set = {}          # 데이터가 담길 dictionary







print("### url 수집 ###")

'''1. 날짜 하루씩 추가하면서 크롤링 진행'''
# '날짜 지정'에서 설정한 검색종료 날짜까지 작업 반복
for day in tqdm(range(number_of_days + 1)) :
    
    # current date (datetime / str )
    current_date_dt = first_date_dt + dt.timedelta(days = day)  # 처음 날짜에서 day만큼 진행
    current_date_str = current_date_dt.strftime('%Y%m%d')
    '''
    2. current_date 를 기반으로 날짜 변수를 datetime 형태로 저장  -
    ->    이후의 날짜 계산을 용이하게 하기 위함
    '''
    s_date = current_date_dt 
    e_date = s_date + dt.timedelta(days=1)   


    '''
    3. datetime 날짜의 type 조작   -->    검색창에 입력할 수 있는 형태로 변환
    '''

    # 1.  20200201  형태로 저장
    str_s_date1 = s_date.strftime('%Y%m%d')
    str_e_date1 = e_date.strftime('%Y%m%d')

    # 2,  2020.02.01 형태로 저장
    str_s_date2 = s_date.strftime('%Y.%m.%d')
    str_e_date2 = e_date.strftime('%Y.%m.%d')


    '''
    4. url 및 검색 parameter 지정
    '''

    # base url
    base_url = 'https://search.naver.com/search.naver'

    # parameters
    query_params = '?' + urlencode(
        {
            quote_plus('where') : 'news',
            quote_plus('query') : ' "웹보드게임" | "웹 보드게임" ',
            quote_plus('sm') : 'tab_opt',
            quote_plus('') : '',
            quote_plus('sort')  : '2',
            quote_plus('photo') : '0',
            quote_plus('field') : '0',
            quote_plus('pd') : '3',
            quote_plus('ds') : str_s_date2,
            quote_plus('de') : str_e_date2,
            quote_plus('docid') : '',  
            quote_plus('related') : '0',
            quote_plus('mynews') : '0',       # 언론사 선택시 1 / default = 0 
            quote_plus('office_type') :  '0',  # default = 0 / 일간지 = 1 
            quote_plus('office_section_code') : '0',  # 언론사 선택하면 1, default = 0
            
            ########################
            ### 신문사 코드 입력 ###
            ########################
            
            quote_plus('news_office_checked') : '',  # 언론사 선택 안할 땐 공백으로 비워두기
        }
    )


    # 첫번째 반복시행에서 article index는 1 고정
    first_article_index = 1




    ###############################  특정 날짜에서의 검색창 크롤링 ###########################
    #iteration_idx = 0
    while True:    
        try:
            '''
            5. 뉴스 검색창 접속
            '''
            # 기타 parameter
            start = '&start=' + str(first_article_index)  # 페이지 첫번째 기사의 index, 
                                    # 처음은 1로 시작하나 페이지 넘길 때마다 10씩 증가시켜야 함
                                    # 즉, 날짜별 iteration 코드에 < start += 10 >  를 넣어주어야 함
            query_others = '&nso=so%3Ar%2Cp%3Afrom' + str_s_date1 + 'to' + str_e_date1 + '&is_sug_officeid=0' + start

            # url
            url = base_url + query_params + query_others
            # 서버에 접속. 너무 급한 속도의 접속은 IP ban을 유발할 수 있음
            req = Request(url, headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
            )
            page = urlopen(req)
            
        except:
            t.sleep(20)

            '''
            5. 뉴스 검색창 접속
            '''
            # 기타 parameter
            start = '&start=' + str(first_article_index)  # 페이지 첫번째 기사의 index, 
                                    # 처음은 1로 시작하나 페이지 넘길 때마다 10씩 증가시켜야 함
                                    # 즉, 날짜별 iteration 코드에 < start += 10 >  를 넣어주어야 함
            query_others = '&nso=so%3Ar%2Cp%3Afrom' + str_s_date1 + 'to' + str_e_date1 + '&is_sug_officeid=0' + start

            # url
            url = base_url + query_params + query_others
            # 서버에 접속. 너무 급한 속도의 접속은 IP ban을 유발할 수 있음
            req = Request(url, headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
            )
            page = urlopen(req)

            
            
        t.sleep(0.3)     # IP ban 방지용
        

        
        
        '''
        6. page parsing
        '''
        
        # bs parsing
        page = bs(page, 'html.parser')


        # IS_EMPTY 는 검색결과가 있다면 [] 를 반환함. 
        # 만일 IS_EMPTY 가 []가 아니라면 검색결과가 없는 것이므로 break 해야 함.
        IS_EMPTY = page.find_all('div', 'api_noresult_wrap')
        if IS_EMPTY != []:
            break 

        # 필요 데이터 parsing (url, 날짜)
        url_resultset = page.find_all('div', 'news_info')   #  --> url
        news_date_resultset = page.find_all('div', 'info_group')  # --> 날짜,  둘다 parsing 후의 index는 같음



        '''
        7. 지면뉴스 오류 handling
        '''
        for idx in range(len(url_resultset)):
            # url
            target_url = url_resultset[idx].a['data-url']

            # 날짜
            if news_date_resultset[idx].i:
                IS_PAPERNEWS = True
            else:
                IS_PAPERNEWS = False

            if IS_PAPERNEWS ==False:
                target_date = news_date_resultset[idx].find('span', 'info').text

            else:
                try:
                    target_date = news_date_resultset[idx].find_all('span', 'info')[1].text
                except:
                    target_date = news_date_resultset[idx].find_all('span', 'info')[0].text


            # 데이터 저장, key = url  / value = date
            data_set[target_url] = target_date




        # 다음 검색 페이지로 넘어가기 위해 기사 index 조정
        first_article_index += 10



'''
데이터 내보내기 (url)
'''     
        
table = pd.DataFrame(data_set.keys(), data_set.values())
table = table.drop_duplicates()
table.to_csv('전체기사 url주소.csv' , encoding = 'cp949')    
    


    
    
    


################################################
end_time = t.time()
duration = round((end_time - start_time)/60, 2)


print('작업 완료!')
print('작업 소요시간 : 약 {}분'.format(duration))
print('작업 완료 시간 : ', str(dt.datetime.now()).split(' ')[0], str(dt.datetime.now()).split(' ')[1].split('.')[0])


### url 수집 ###


100%|██████████| 386/386 [07:15<00:00,  1.13s/it]

작업 완료!
작업 소요시간 : 약 7.26분
작업 완료 시간 :  2021-09-14 10:54:55
